# Pregunta 3

In [1]:
import cPickle as pickle
import numpy as np
import os

def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [2]:
def load_CIFAR_one(filename):
    with open(filename, 'rb') as f:
        datadict = pickle.load(f)
        X = datadict['data']
        Y = datadict['labels']
        return X, np.array(Y)
    
def load_CIFAR10(PATH, n_val=10000):
    if n_val > 10000:
        n_val = 10000
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_one(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    Xval = Xtr[-n_val:Xtr.shape[0]]
    Yval = Ytr[-n_val:Ytr.shape[0]]
    Xtr = Xtr[:-n_val]
    Ytr = Ytr[:-n_val]
    del X, Y
    Xte, Yte = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
    return Xtr, Ytr, Xval, Yval, Xte, Yte

In [23]:
Xtr, Ytr, Xval, Yval, Xte, Yte = load_CIFAR10('.')

In [26]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

def scaler_function(Xtr,Xval,Xt,other=True, scale=True):
    if other:
        Xtr_scaled = Xtr / 255.
        Xval_scaled = Xval / 255.
        Xt_scaled = Xt / 255.
        return Xtr_scaled, Xval_scaled, Xt_scaled
    scaler = StandardScaler(with_std=scale).fit(Xtr)
    Xtr_scaled = scaler.transform(Xtr)
    Xval_scaled = scaler.transform(Xval)
    Xt_scaled = scaler.transform(Xt)
    return Xtr_scaled, Xval_scaled, Xt_scaled

In [5]:
"""%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(1,1))
sample = Xtr[12].reshape(3,32,32).T
plt.imshow(sample, interpolation="nearest")
plt.show()

Xtr_scaled, Xt_scaled = scaler_function(Xtr, Xte, other=True)
plt.figure(figsize=(1,1))
sample = Xtr_scaled[12].reshape(3,32,32).T
plt.imshow(sample, interpolation="nearest")
plt.show()

Xtr_scaled, Xt_scaled = scaler_function(Xtr.astype(np.float64), Xte.astype(np.float64), other=False, scale=False)
plt.figure(figsize=(1,1))
sample = Xtr_scaled[12].reshape(3,32,32).T
plt.imshow(sample, interpolation="nearest")
plt.show()

Xtr_scaled, Xt_scaled = scaler_function(Xtr.astype(np.float64), Xte.astype(np.float64), other=False, scale=True)
plt.figure(figsize=(1,1))
sample = Xtr_scaled[12].reshape(3,32,32).T
plt.imshow(sample, interpolation="nearest")
plt.show()"""

'%matplotlib inline\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(1,1))\nsample = Xtr[12].reshape(3,32,32).T\nplt.imshow(sample, interpolation="nearest")\nplt.show()\n\nXtr_scaled, Xt_scaled = scaler_function(Xtr, Xte, other=True)\nplt.figure(figsize=(1,1))\nsample = Xtr_scaled[12].reshape(3,32,32).T\nplt.imshow(sample, interpolation="nearest")\nplt.show()\n\nXtr_scaled, Xt_scaled = scaler_function(Xtr.astype(np.float64), Xte.astype(np.float64), other=False, scale=False)\nplt.figure(figsize=(1,1))\nsample = Xtr_scaled[12].reshape(3,32,32).T\nplt.imshow(sample, interpolation="nearest")\nplt.show()\n\nXtr_scaled, Xt_scaled = scaler_function(Xtr.astype(np.float64), Xte.astype(np.float64), other=False, scale=True)\nplt.figure(figsize=(1,1))\nsample = Xtr_scaled[12].reshape(3,32,32).T\nplt.imshow(sample, interpolation="nearest")\nplt.show()'

In [24]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [28]:
Xtr, Xval, Xte = scaler_function(Xtr, Xval, Xte, other=False)
Ytr_cat = to_categorical(Ytr, 10)
Yval_cat = to_categorical(Yval, 10)
Yte_cat = to_categorical(Yte, 10)

/usr/lib64/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
model = Sequential()
model.add(Dense(100, input_dim=Xtr.shape[1], init='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, init='uniform', activation='softmax'))
model.compile(optimizer=SGD(lr=0.05), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(Xtr, Ytr_cat, nb_epoch=50, batch_size=32, verbose=1, validation_data=(Xval,Yval_cat))
scores = model.evaluate(Xval, Yval_cat,)
test_acc = scores[1]

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 5s - loss: 0.2735 - acc: 0.9028 - val_loss: 2.2219 - val_acc: 0.8475
Epoch 2/50
40000/40000 [==============================] - 5s - loss: 0.2486 - acc: 0.9074 - val_loss: 2.1756 - val_acc: 0.8516
Epoch 3/50
40000/40000 [==============================] - 5s - loss: 0.2386 - acc: 0.9103 - val_loss: 2.0899 - val_acc: 0.8582
Epoch 4/50
40000/40000 [==============================] - 5s - loss: 0.2321 - acc: 0.9124 - val_loss: 1.9405 - val_acc: 0.8666
Epoch 5/50
40000/40000 [==============================] - 5s - loss: 0.2272 - acc: 0.9141 - val_loss: 2.0570 - val_acc: 0.8614
Epoch 6/50
40000/40000 [==============================] - 5s - loss: 0.2223 - acc: 0.9156 - val_loss: 1.8752 - val_acc: 0.8699
Epoch 7/50
40000/40000 [==============================] - 5s - loss: 0.2182 - acc: 0.9171 - val_loss: 2.0068 - val_acc: 0.8642
Epoch 8/50
40000/40000 [==============================] - 5s 

KeyboardInterrupt: 